# Storm tracking in ERA5 data

*Author : Frédéric FERRY - Météo-France / Ecole Nationale de la Météorologie (June 2023)*

Concepts illustrated here :
- Storm tracking from ERA5 data in netcdf format (mean sea level pressure)

ERA5 data in netcdf format can be downloaded here:
- https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form

In [2]:
import os

import xarray as xr
import netCDF4
import numpy as np
import pandas as pd

from shapely.geometry import Point

from cartopy import config
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid

from scipy.ndimage import maximum_filter, minimum_filter

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob
from tqdm import tqdm

In [ ]:
def plot_maxmin_points(data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values, symbol, color=color, size=12,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values,
                '\n' + str(int(data[mxy[i], mxx[i]])),
                color=color, size=10, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

def print_maxmin_points(data, extrema, nsize):
    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        # Print date lon lat and pressure of the minimum
        print(date, data.longitude[mxx[i]].values, data.latitude[mxy[i]].values, int(data[mxy[i], mxx[i]]))

In [ ]:
def make_animation(gif_filepath):
    from PIL import Image
    import os
    from IPython.display import Image as IPImage
    from IPython.display import display
    import time
    
    image_folder = './anim/'+storm+'/' # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    for filename in image_files:
        os.remove(image_folder+filename)

In [ ]:
projection=ccrs.PlateCarree()

def plot_background(ax):
    ax.coastlines()
    ax.gridlines()
    ax.set_xticks(np.linspace(-180, 180, 19), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-90, 90, 19), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    return(ax)             

# Open and plot MSLP data

In [ ]:
storm='Zorbas'
dir_data="./data/"+storm+'/'
    
if not os.path.exists('./anim/'+storm):
    os.mkdir('./anim/'+storm)

if not os.path.exists('./figs/'+storm):
    os.mkdir('./figs/'+storm)

if not os.path.exists('./figs/'+storm):
    os.mkdir('./figs/'+storm)

dir_anim='./anim/'+storm+'/'

In [ ]:
msl_era    = xr.open_dataset(dir_data+"msl.nc")
print(msl_era)

<div class="alert alert-danger">
<p><b>1) Reduce the available data to September 27th 00UTC - September 30th 18UTC (the date format must be 'YYYY-MM-DDTXX').</b></p>
<p><b>2) Reduce the geographical domain to 25N-70N 70W-35E (the lat/lon must be numbers).</b></p>

</div>

In [ ]:
date1=''
date2=''

latS=
latN=
lonW=
lonE=

In [ ]:
msl_era    = xr.open_dataset(dir_data+"msl.nc").sel(
    time=slice(date1,date2)).sel(
    latitude=slice(latN,latS)).sel(
    longitude=slice(lonW,lonE))

print(msl_era)

In [ ]:
mslp = msl_era['msl']/100
lat  = mslp.latitude.values
time  = mslp.time.values
lon  = mslp.longitude.values

print(mslp.shape)

In [ ]:
mslp_levels = np.arange(970,1042,2)

for i in tqdm(range(len(time))):
    #print(str(time[i])[0:13])
    fig = plt.figure(figsize=(15., 10.))

    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('MSLP : '+str(time[i])[0:13],loc='center',fontsize=14)
    plot_background(ax)
    
    # MSLP in contours and min max
    c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="grey", linewidths=1, transform=ccrs.PlateCarree())
    #ax.clabel(c1,fmt='%4.1i',fontsize=10)
    cf = ax.contourf(lon, lat, mslp[i,:,:], levels=mslp_levels, cmap='jet', extend='both', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
    plot_maxmin_points(mslp[i,:,:], 'min', 25,
                       symbol='L', color='b', transform=ccrs.PlateCarree())    
    figname='./anim/'+storm+'/MSL_'+str(time[i])[0:13]
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
gif_filepath = './anim/'+storm+'/MSL.gif'
make_animation(gif_filepath)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Locate tropical cyclone Leslie over the Atlantic. Is the system strengtening or weakening ? </p>
<p><b>2) </b>Locate medicane Zorbas between Lybia and Greece and indicate its initation time.</p>
</div>

<div class="alert alert-success">
<b> </b>
</div>

# Tracking medicane Zorbas

<div class="alert alert-danger">
<p><b>Choose the tracking period of the medicane (the date format must be 'YYYY-MM-DDTXX')</b></p>
</div>

In [ ]:
date1=''
date2=''

In [ ]:
msl_era    = xr.open_dataset(dir_data+"msl.nc").sel(
    time=slice(date1,date2)).sel(
    latitude=slice(latN,latS)).sel(
    longitude=slice(lonW,lonE))

print(msl_era)

In [ ]:
mslp = msl_era['msl']/100
lat  = mslp.latitude.values
time  = mslp.time.values
lon  = mslp.longitude.values

print(mslp.shape)

In [ ]:
%%capture cap --no-stderr
for i in range(len(time)):
    date=str(time[i])[0:13]   
    print_maxmin_points(mslp[i,:,:], 'min', 25)

In [ ]:
file_era = './txt/era5_tracking.txt'
with open(file_era, 'w') as f:
    f.write(cap.stdout)
f.close()

In [ ]:
def tracking(file): 
    df = pd.read_csv(file,sep=" ",header=None)
    liste_date =  np.unique(df[0].values)
    # We will track lows that are present at initial time
    ds = df.to_xarray()
    original_position = ds.sel(index= ds[0] == liste_date[0])
    # Build individual tracking for each detected low
    traj = []
    for ind in original_position.index.values: # Boucle sur les depressions à t=0
        position = Point(original_position.sel(index=ind)[1],original_position.sel(index=ind)[2])
        dep_traj = []
        dep_traj.append((
            original_position.sel(index=ind)[1].values,
            original_position.sel(index=ind)[2].values,
            original_position.sel(index=ind)[0].values,
            original_position.sel(index=ind)[3].values, 
            ))
        for date in liste_date[1:]: # Loop for all timesteps
            temp = ds.sel(index= ds[0] == date)
            l_area = []
            for other_idx in temp.index.values: 
                oth_pos = Point(temp.sel(index=other_idx)[1],temp.sel(index=other_idx)[2])
                l_area.append(position.buffer(1).intersection(oth_pos.buffer(1)).area) 
                # Finding intersection between two circles of 1°. 
            if np.max(l_area)>0.001: # Compare areas
                elt = np.argmax(l_area)
                n_position=Point(temp.isel(index=elt)[1],temp.isel(index=elt)[2])
                dep_traj.append([
                    temp.isel(index=elt)[1].values,
                    temp.isel(index=elt)[2].values,
                    temp.isel(index=elt)[0].values,
                    temp.isel(index=elt)[3].values] )
                position = n_position
            else: 
                break
        traj.append(dep_traj)
    return traj

In [ ]:
tracking_era = tracking(file_era)
print("Number of tracked lows : ", len(tracking_era))

<div class="alert alert-danger">
<p><b>Select the approximate lon/lat of the medicane at the initial time.</b></p>
</div>

In [ ]:
lon_init = 
lat_init = 

In [ ]:
fig = plt.figure(figsize=(15., 10.))

ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('MSLP : '+str(date1),loc='center',fontsize=14)
plot_background(ax)
ax.set_extent([5, 35, 30, 45])
# MSLP in contours and min max
c1 = ax.contour(lon, lat, mslp[0,:,:], levels=mslp_levels, colors="black", linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c1,fmt='%4.1i',fontsize=10)
plot_maxmin_points(mslp[0,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())
ax.scatter(lon_init,lat_init, c='green', transform=ccrs.PlateCarree())
plt.show()

In [ ]:
def selection(lon_init,lat_init,tracking): 
    dist_min = 1000
    ind= -1
    for dep in range(len(tracking)): 
        dist = (tracking[dep][0][0] - lon_init)**2+(tracking[dep][0][1] - lat_init)**2
        if dist < dist_min: 
            dist_min = dist 
            ind = dep 
    return ind

In [ ]:
ind_era = selection(lon_init,lat_init,tracking_era)
print(tracking_era[ind_era])
print(len(tracking_era[ind_era]))

In [ ]:
def get_list(low): 
    list_lat = []
    list_lon = []
    list_time = []
    list_pres = []
    for i in range(len(low)):
        list_lon.append(low[i][0])
        list_lat.append(low[i][1])
        list_time.append(low[i][2])
        list_pres.append(low[i][3])
    return list_lon, list_lat, list_time, list_pres

In [ ]:
if ind_era >=0:
    liste_lon, liste_lat, liste_time, liste_pres = get_list(tracking_era[ind_era])
    list_time = [str(x) for x in liste_time]

In [ ]:
file_storm = './txt/Zorbas.txt'
file=open(file_storm, "w+")
for i in range(len(tracking_era[ind_era])):
    file.write(str(liste_time[i]))
    file.write(' ')
    file.write(str(liste_lon[i]))
    file.write(' ')
    file.write(str(liste_lat[i]))
    file.write(' ')
    file.write(str(liste_pres[i]))
    file.write("\n")
file.close()

In [ ]:
for i in tqdm(range(len(tracking_era[ind_era]))):
    #print(str(tracking_era[ind_era][i][2])[0:13])
    fig = plt.figure(figsize=(15., 10.))

    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('MSLP : '+str(tracking_era[ind_era][i][2])[0:13],loc='center',fontsize=14)
    plot_background(ax)
    ax.set_extent([5, 35, 30, 45])
    
    # MSLP in contours and min max
    c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="black", linewidths=1, transform=ccrs.PlateCarree())
    ax.clabel(c1,fmt='%4.1i',fontsize=10)
    plot_maxmin_points(mslp[i,:,:], 'min', 25,
                       symbol='L', color='b', transform=ccrs.PlateCarree()) 
    # Track from lat/lon in txt file
    ax.scatter(tracking_era[ind_era][i][0],tracking_era[ind_era][i][1], c='green', transform=ccrs.PlateCarree())
    ax.plot(liste_lon[0:i+1],liste_lat[0:i+1], c='red', marker='+', transform=ccrs.PlateCarree())
    
    figname='./anim/'+storm+'/MSL_tracking_'+str(time[i])[0:13]
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
gif_filepath = './anim/'+storm+'/MSL_tracking.gif'
make_animation(gif_filepath)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Does the automatic tracking method successfully track medicane Zorbas ?</p>
<p><b>2) </b>Describe the track of medicane Zorbas.</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

<div class="alert alert-danger">
<p><b>1) Plot a map of the full track of the system with a 6-hourly timestep.</b></p>
<p><b>2) Plot a curve of the MSLP evolution of the system with a 6-hourly timestep.</b></p>
</div>

# Extra Task : track Huricane Leslie over the Atlantic

<div class="alert alert-danger">

<p><b>1) Extract MSLP data from September 24th 00UTC to October 10th 18UTC with a 6-hourly timestep (80 values).</b></p>

<p><b>2) Track Huricane Leslie in the extracted MSLP data and create a Leslie.txt tracking file (time/lon/lat/pres) in the ./txt folder.</b></p>

<p><b>3) Verify your tracking with animations or maps.</b></p>

<br>

    
<p><b>Hint 1 : the pandas date_range function may be useful https://pandas.pydata.org/docs/reference/api/pandas.date_range.html.</b></p>
<p><b>Hint 2 : you will have to change position.buffer(1) to position.buffer(6) in the automatic tracking function.</b></p>


</div>